In [4]:
from Datos import Datos
from Arm import ArmkNN
import numpy as np

from sklearn.model_selection import train_test_split

dataset = Datos("ratings.csv")

kNNcols = ['userId','movieId','rating']
kNNdataset = dataset.extraeCols(kNNcols)


listUsers = np.unique(kNNdataset[:,0])
knn = ArmkNN(20)


In [5]:
knn.rec_item(1)

3275.0

In [7]:
# Prueba de kNN
train, test = train_test_split(kNNdataset, train_size=0.1)
knn.initSet(train)

num = 0
den = 0

for u in listUsers:
    item = knn.rec_item(u)
    # Ejemplo encontrado para la recomendación
    mask1 = test[:,0] == u
    mask2 = test[:,1] == item
    mask = np.logical_and(mask1,mask2)
    test, hit = test[np.logical_not(mask)], test[mask]
    if(len(hit) != 0):
        train = np.vstack((train,hit))
        den += 1
        if(hit[0,2] >= 3):
            num += 1
        print(num/den)
    else:
        train = np.vstack((train,[u,item,1.]))

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9803921568627451
0.9807692307692307
0.9811320754716981
0.9814814814814815
0.9818181818181818
0.9821428571428571
0.9824561403508771
0.9827586206896551
0.9830508474576272
0.9833333333333333
0.9836065573770492
0.9838709677419355
0.9841269841269841
0.984375
0.9846153846153847
0.9848484848484849
0.9850746268656716
0.9852941176470589
0.9855072463768116
0.9857142857142858
0.9859154929577465
0.9861111111111112
0.9863013698630136
0.9864864864864865
0.9866666666666667
0.9868421052631579
0.987012987012987
0.9871794871794872
0.9873417721518988
0.9875
0.9876543209876543
0.9878048780487805
0.9879518072289156
0.9880952380952381
0.9882352941176471
0.9883720930232558
0.9885057471264368
0.9886363636363636
0.9887640449438202
0.9888888888888889
0.989010989010989
0.9891304347826086
0.989247311827957
0.989361

Pruebas de NB

In [3]:
from Datos import Datos
from Arm import ArmNB
import numpy as np

from sklearn.model_selection import train_test_split


dataset = Datos("ratings.csv")

kNNcols = ['userId','movieId','rating']
kNNdataset = dataset.extraeCols(kNNcols)

train, test = train_test_split(kNNdataset, train_size=0.1)

arm = ArmNB()
arm.initSet(train)

arm.rec_item(1)

6753.0

In [10]:
from Datos import Datos
from Arm import ArmNB
import numpy as np

from sklearn.model_selection import train_test_split

dataset = Datos("ratings.csv")

kNNcols = ['userId','movieId','rating']
kNNdataset = dataset.extraeCols(kNNcols)


listUsers = np.unique(kNNdataset[:,0])
train, test = train_test_split(kNNdataset, train_size=0.1)
arm = ArmNB(train)

In [11]:
# Prueba de NB


num = 0
den = 0

for u in listUsers:
    item = arm.rec_item(u)
    # Ejemplo encontrado para la recomendación
    mask1 = test[:,0] == u
    mask2 = test[:,1] == item
    mask = np.logical_and(mask1,mask2)
    test, hit = test[np.logical_not(mask)], test[mask]
    if(len(hit) != 0):
        train = np.vstack((train,hit))
        den += 1
        if(hit[0,2] >= 3):
            num += 1
        print(num/den)
    else:
        train = np.vstack((train,[u,item,1.]))

1.0
1.0
1.0
0.75
0.8
0.8333333333333334
0.8571428571428571
0.875
0.8888888888888888
0.9
0.9090909090909091
0.9166666666666666
0.9230769230769231
0.9285714285714286
0.9333333333333333
0.9375
0.9411764705882353
0.9444444444444444
0.9473684210526315
0.9
0.9047619047619048
0.9090909090909091
0.9130434782608695
0.9166666666666666
0.92
0.9230769230769231
0.9259259259259259
0.9285714285714286
0.9310344827586207
0.9
0.9032258064516129
0.90625
0.9090909090909091
0.9117647058823529
0.9142857142857143
0.9166666666666666
0.918918918918919
0.9210526315789473
0.9230769230769231
0.925
0.926829268292683
0.9285714285714286
0.9302325581395349
0.9318181818181818
0.9333333333333333
0.9347826086956522
0.9361702127659575
0.9375
0.9387755102040817
0.94
0.9411764705882353
0.9423076923076923
0.9433962264150944
0.9444444444444444
0.9454545454545454
0.9464285714285714
0.9473684210526315
0.9482758620689655
0.9491525423728814
0.95
0.9508196721311475
0.9516129032258065
0.9523809523809523
0.953125
0.9538461538461539

Probando cosas más chungas

In [1]:
from Datos import Datos
from Arm import ArmItemNB
import numpy as np

from sklearn.model_selection import train_test_split

dataset = Datos("ratings.csv")

kNNcols = ['userId','movieId','rating']
kNNdataset = dataset.extraeCols(kNNcols)

datasetM = Datos("movies.csv",nominal=False)

cols = ['movieId','genres']
npdatasetM = datasetM.extraeCols(cols)

train, test = train_test_split(kNNdataset, train_size=0.1)
listUsers = np.unique(train[:,0])

arm = ArmItemNB(npdatasetM)
arm.initSet(train)

In [2]:
# Prueba de NB con items


num = 0
den = 0

for u in listUsers:
    item = arm.rec_item(u)
    # Ejemplo encontrado para la recomendación
    mask1 = test[:,0] == u
    mask2 = test[:,1] == item
    mask = np.logical_and(mask1,mask2)
    test, hit = test[np.logical_not(mask)], test[mask]
    if(len(hit) != 0):
        den += 1
        if(hit[0,2] >= 3):
            rt = 1
            num += 1
        else:
            rt = -1
        print(num/den)
        arm.trainSet = np.vstack((arm.trainSet,[u,item,rt]))
    else:
        arm.trainSet = np.vstack((arm.trainSet,[u,item,-1]))

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9090909090909091
0.9166666666666666
0.8461538461538461
0.8571428571428571
0.8666666666666667
0.875
0.8823529411764706
0.8888888888888888
0.8421052631578947
0.8
0.8095238095238095
0.8181818181818182
0.8260869565217391
0.8333333333333334
0.84
0.8461538461538461
0.8148148148148148
0.8214285714285714
0.8275862068965517
0.8333333333333334
0.8387096774193549
0.84375
0.8484848484848485
0.8529411764705882
0.8571428571428571
0.8611111111111112
0.8648648648648649
0.868421052631579
0.8717948717948718
0.875
0.8780487804878049
0.8809523809523809
0.8837209302325582
0.8636363636363636
0.8666666666666667
0.8695652173913043
0.8723404255319149
0.875
0.8775510204081632
